In [1]:
# Google Sheets API Libraries
from __future__ import print_function
import pickle
import os.path
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
import gspread
from oauth2client.service_account import ServiceAccountCredentials
 
# -----------------------------------------------------------------------#

import pandas as pd
import pprint as pp

# -----------------------------------------------------------------------#

# Telegram Bot API Libraries
from telepot.loop import MessageLoop
from telepot.namedtuple import InlineKeyboardMarkup, InlineKeyboardButton
import telepot

In [ ]:
# 3 Inputs:
# Credentials - Basically the credentials.json file
# Sheetname - The name of your spreadsheet on Google Sheets
# Range - 'A1:Z99': For you to customize, for example, you have a header row for columns, you wanna exclude that
def opener(credentials, sheetname):
    # Scope means what you want to have access to
    # First we have access to google sheets API
    # Then we have access to google drive API
    scope = ['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive']
 
    # Credentials is the file you downloaded
    credentials = ServiceAccountCredentials.from_json_keyfile_name(credentials, scope)

    # Authorize your credentials
    gc = gspread.authorize(credentials)

    # Opening your google sheets\n,
    # gc.open('sheetname').sheet1/2/3/4\n,
    sheet = gc.open(sheetname).sheet1
    
    # To specify the range of the final sheet produced
    # final_sheet = sheet.values_get(sheetrange)

    return sheet

In [ ]:
# Helper Functions
def opener(credentials, sheetname):
    scope = ['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive']
 
    credentials = ServiceAccountCredentials.from_json_keyfile_name(credentials, scope)

    gc = gspread.authorize(credentials)

    sheet = gc.open(sheetname).sheet1
    
    return sheet

# Variables Store
# Google Sheets Id
sheetid = # Input your Google Sheet Id here in the format of 'sheetid'

# Bot Token
Token = # Input your Telegram Bot token here in format of 'bot token'

# Sheet Credentials
CRED = # Input the json file name of your sheet credentials here in format of'filename.json'

# Sheet Name
SNAME = # Input the name of your Google Spreadsheet here in format of 'sheetname'

# Step variables
add_step = 0
r_step = 0
d_step = 0
u_step = 0
# -----------------------------------------------------------------------#

# Text Libraries
welcome_text = 'Welcome to Password Manager Bot.\n\n\n\nTo add a new password entry, please type /addnew\nTo retrieve a password entry, please type /retrieve\nTo delete an existing entry, please type /delete\nTo update an existing password entry, please type /update'

# /addnew text
add_new1 = 'Are you sure you want to add a new password entry? Enter /yes to confirm.'
add_new2 = 'Please enter the website that this password is used for - E.g. Hotmail'
add_new3 = 'Please enter your username.'
add_new4 = 'Please enter your password.'

# /retrieve text
r_text1 = 'Please enter the website which you which to retrieve your password for.\n\nUse /listall to view all websites that you have stored your password on.\n'

# /listall text
listall1 = 'Your Passwords are stored on the following websites:'

# /delete text
d_text1 = 'Please enter the website of the password entry which you wish to delete.\n\nUse /listall to view all websites that you have stored your password on.\n'

# /update text
u_text1 = 'Please enter the website of the password entry which you wish to update.\n\nUse /listall to view all websites that you have stored your password on.\n'
# -----------------------------------------------------------------------#

# Handlers
# Text Handler
def on_chat_message(message):
    content_type, chat_type, chat_id = telepot.glance(message)
    global add_step, r_step, d_step, u_step, username, password, website, r_web, u_user, u_pass, u_site
    
    # ----- WELCOME ----- # 
    # Print welcome message when user inputs /start
    if message['text'] == '/start':
        bot.sendMessage(chat_id, welcome_text)
    
    
    # ----- ADD NEW ENTRY ----- # 
    # Start of process of adding new password entry into Google Sheets
    if message['text'] == '/addnew':
        add_step = 1
        bot.sendMessage(chat_id, add_new1)
    # If user inputs /yes, then will proceed to next step which is to prompt for the website    
    elif message['text'] == '/yes' and add_step==1:
        add_step = 2
        bot.sendMessage(chat_id, add_new2)
    # User inputs website, then bot prompts for username 
    # Previous user input is saved under the website variable
    # Move on to next step after prompting for username
    elif add_step==2:
        website = message['text']
        bot.sendMessage(chat_id, add_new3)
        add_step = 3
    # User input is saved under the username variable
    # Bot prompts for the password then moves on to the next step
    elif add_step == 3:
        username = message['text']
        bot.sendMessage(chat_id, add_new4)
        add_step = 4
    # User input is saved under the password variable
    # Bot then prompts for confirmation, printing out all the previous user input
    # Move on to next step after prompt
    elif add_step == 4:
        password = message['text']
        bot.sendMessage(chat_id, 'Type /yes to confirm the following:\n' + 'Website is: ' + website + '\n' + 'Username is: ' + username + '\n' + 'Password is: ' + password + '\n\n' + 'If it is incorrect, type /no to start over.')
        add_step = 5
    # If user confirms information is correct with /yes, then bot tells user information is saved to Google Sheets
    # Open google sheets, then add a new row containing all the information previously input by user
    elif message['text'] == '/yes' and add_step == 5:
        bot.sendMessage(chat_id, 'Ok, your entry has been saved onto Google Sheets for future reference.')
        sht = opener(CRED, SNAME)
        sht.append_row([website, username, password], value_input_option='RAW')
        add_step = 0
    # If user answers with /no, bot wil prompt user to start over
    elif message['text'] == '/no' and add_step == 5:
        bot.sendMessage(chat_id, 'Please type /addnew to start over.')
        add_step = 0
    
    
    # ----- RETRIEVE PASSWORD ----- #
    # If user inputs /retrieve bot will send prompt asking user what website's password user wishes to retrieve
    # Move on to r_step 1
    if message['text'] == '/retrieve':
        bot.sendMessage(chat_id, r_text1)
        r_step = 1
    # Saves user input as r_web
    # Uses r_web as a search value for the Google Sheets using gspread
    # .find() will produce a gpsread cell object, which has .row that can tell you which row the value is located on
    # Then use .row_values(cell.row)[N] to extract out the website, username and password from Google Sheets
    # Bot will these information back to the user, then r_step is resetted back to 0
    elif r_step == 1:
        r_web  = message['text']
        sht = opener(CRED, SNAME)
        cell = sht.find(r_web)
        web = sht.row_values(cell.row)[0]
        user = sht.row_values(cell.row)[1]
        pword = sht.row_values(cell.row)[2]
        reply = 'For ' + web + ':\n' + 'Username is: ' + user +'\n' + 'Password is: ' + pword
        bot.sendMessage(chat_id, reply)
        r_step = 0
    
    
    # ----- LIST ALL ENTRIES ----- #
    # If user inputs /listall, then bot will send a list of all the websites user stored their password on
    # Use gspread to pull the data using .get_all_records(), then store in a list called sites, then bot send out sites
    if message['text'] == '/listall':
        sht = opener(CRED, SNAME)
        bot.sendMessage(chat_id, listall1)
        sites= []
        for values in sht.get_all_records():
            sites.append([values['Website']])
        bot.sendMessage(chat_id, sites)
    
    
    # ----- DELETE AN ENTRY ----- #
    # If user inputs /delete, bot will start the process for deleting an entry
    # Bot will ask user which website they wish to delete it from
    # d_step = 1 to proceed to next step
    if message['text'] == '/delete':
        bot.sendMessage(chat_id, d_text1)
        d_step = 1
    # User input is saved as d_web which will be used to find the row that contains this website
    # Then entire row is deleted with .delete_row, bot then sends a message telling user that the entry has been deleted
    # d_step resets to 0
    elif d_step == 1:
        d_web = message['text']
        sht = opener(CRED, SNAME)
        cell = sht.find(d_web)
        sht.delete_row(cell.row)
        reply = 'Your entry for ' + d_web + ' has been deleted from Google Sheets.' 
        bot.sendMessage(chat_id, reply)
        d_step = 0
    
    
    # ----- UPDATE AN ENTRY ----- #
    # If user inputs /update, bot will prompt user asking for the website of entry that they wanna update
    # u_step = 1 to move on to next step
    if message['text'] == '/update':
        bot.sendMessage(chat_id, u_text1)
        u_step = 1
    # User input website is saved as u_site
    # Bot prompts for updated username
    # Move on to next step with u_step = 2
    elif u_step == 1:
        u_site = message['text']
        bot.sendMessage(chat_id, 'Please enter your updated username.')
        u_step = 2
    # User input username is saved as u_user
    # Bot prompts for updated password
    # Move on to next step with u_step = 3
    elif u_step == 2:
        u_user = message['text']
        bot.sendMessage(chat_id, 'Please enter your updated password.')
        u_step = 3
    # User input password is saved as u_pass
    # Bot prompts for user to confirm that the previously entered information is correct
    # Move on to next step with u_step = 4
    elif u_step == 3:
        u_pass = message['text']
        bot.sendMessage(chat_id, 'Type /yes to confirm the following:\n' + 'Updated username is: ' + u_user + '\n' + 'Updated password is: ' + u_pass + '\n\n' + 'If it is incorrect, type /no to start over.')
        u_step = 4
    # If user inputs /yes, then will use user entered website to find the row that contains that information with sht.find()
    # Then will update the username & password with .update_cell()
    # Bot then sends message to user that information has been updated
    # u_step resetted to 0
    elif message['text'] == '/yes' and u_step == 4:
        sht = opener(CRED, SNAME)
        cell = sht.find(u_site)
        # Update the username 
        sht.update_cell(cell.row, cell.col+1, u_user)
        # Update the password
        sht.update_cell(cell.row, cell.col+2, u_pass)
        reply = 'Ok, your username and password for ' + u_site + ' has been updated.'
        bot.sendMessage(chat_id, reply)
        u_step = 0
    # If user inputs /no, then bot will prompt user to restart over
    # u_step resetted to 0
    elif message['text'] == '/no' and u_step == 4:
        bot.sendMessage(chat_id, 'Please type /update to start over.')
        u_step = 0
    
        
        
        


# Creating the bot
bot = telepot.Bot(Token)

# MessageLoop to keep running getUpadddates, except in this case
# A dictionary is passed in format of {'source' : handler_function, ...}
MessageLoop(bot, {'chat' : on_chat_message}).run_as_thread()
#                  'callback_query' : on_callback_query}
           
print('Bot is Alive')